In [ ]:
import numpy as np
import tensorflow as tf
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import data_utils
from tqdm import tqdm, trange

In [ ]:
def write_ds_to_tfrecord(dataset, rfrecord_filename):
    with tf.io.TFRecordWriter(rfrecord_filename) as writer:
        for x, y in tqdm(dataset):
            # Serialize the input and output tensors
            x_serialized = tf.io.serialize_tensor(x)
            y_serialized = tf.io.serialize_tensor(y)

            # dictionary representing the example
            example = {
                'x': tf.train.Feature(bytes_list=tf.train.BytesList(value=[x_serialized.numpy()])),
                'y': tf.train.Feature(bytes_list=tf.train.BytesList(value=[y_serialized.numpy()]))
            }

            # create features message using example
            features = tf.train.Features(feature=example)
            example_proto = tf.train.Example(features=features)

            # serialize to string and write example
            writer.write(example_proto.SerializeToString())

In [ ]:
data_path = '../../data/relational_games'

In [ ]:
tasks = ('1task_between', '1task_match_patt')
splits = ('hexos', 'pentos', 'stripes')

filename_prefixes = [f'{task}_{split}' for task in tasks for split in splits]

for filename_prefix in tqdm(filename_prefixes):
    npz_ds = data_utils.load_ds_from_npz(f'{data_path}/{filename_prefix}.npz')
    write_ds_to_tfrecord(npz_ds, f'{data_path}/{filename_prefix}.tfrecord')
    del npz_ds